## DICE - Notebook 1.2 - Dataset Augmentation

<br/>

```
*************************************************************************
**
** 2017 Mai 23
**
** In place of a legal notice, here is a blessing:
**
**    May you do good and not evil.
**    May you find forgiveness for yourself and forgive others.
**    May you share freely, never taking more than you give.
**
*************************************************************************
```

<table style="width:100%; font-size:14px; margin: 20px 0;">
    <tr>
        <td style="text-align:center">
            <b>Contact: </b><a href="mailto:contact@jonathandekhtiar.eu" target="_blank">contact@jonathandekhtiar.eu</a>
        </td>
        <td style="text-align:center">
            <b>Twitter: </b><a href="https://twitter.com/born2data" target="_blank">@born2data</a>
        </td>
        <td style="text-align:center">
            <b>Tech. Blog: </b><a href="http://www.born2data.com/" target="_blank">born2data.com</a>
        </td>
    </tr>
    <tr>
        <td style="text-align:center">
            <b>Personal Website: </b><a href="http://www.jonathandekhtiar.eu" target="_blank">jonathandekhtiar.eu</a>
        </td>
        <td style="text-align:center">
            <b>RSS Feed: </b><a href="https://www.feedcrunch.io/@dataradar/" target="_blank">FeedCrunch.io</a>
        </td>
        <td style="text-align:center">
            <b>LinkedIn: </b><a href="https://fr.linkedin.com/in/jonathandekhtiar" target="_blank">JonathanDEKHTIAR</a>
        </td>
    </tr>
</table>

## Objectives

In order to maximise the robustness of the re-trained model, each image in the dataset will be loaded and augmented.

The augmentation process consists in varying image characteristics such as *brightness, saturation, hue, contrast, gamma, orientation, etc.* These modifications applied to the image are randomly set. 

This process tends to improve the generalisation power of the model. The number of augmented images generated directly impact the training time and the memory requirements, thus leading to a tradeoff between memory, computing power and the model accuracy.

For this study, we have chosen to generate 30 augmented images + the original one, leading to 31 images saved per image in the dataset.


## 1. Load the necessary libraries and initialise global variables

In [1]:
import os, string, random

import tensorflow as tf
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt

%matplotlib inline

################################## GLOBAL NOTEBOOK VARS ##################################

INPUT_DIRECTORY         = "data_cleaned"
OUTPUT_DIRECTORY        = "data_augmented"

IMG_AUGMENTATION_FACTOR = 30 # The number of augmented images generated from the raw image.

############################### RANDOM VALUE GENERATION SEED #############################

SEED                    = 666

## 2. File Queue and Image Reading Process Definition

### 2.1 Define a queue of all the images in "png" in the specific data folder

Make a queue of file names including all the JPEG images files in the relative image directory.

In [2]:
# Get a list of the sub-directories in the INPUT_DIRECTORY
data_directories = [ name for name in os.listdir(INPUT_DIRECTORY) if os.path.isdir(os.path.join(INPUT_DIRECTORY, name)) ]

# We scan all the files in the sub-directories with the extensions given above
all_files = tf.concat(
    [tf.train.match_filenames_once(INPUT_DIRECTORY + "/" + x + "/*.png") for x in data_directories],
    0
)

filename_queue = tf.train.string_input_producer(
    all_files, # Merge the sub-tensors into one
    num_epochs=1,
    seed=SEED,
    shuffle=True
)

### 2.2 Define a queue for random backgrounds 

In [3]:
# Get a list of the sub-directories in the INPUT_DIRECTORY
INPUT_BG_DIRECTORY = os.path.join("data_bg", "cleaned")

bg_directories  = [ name for name in os.listdir(INPUT_BG_DIRECTORY) if os.path.isdir(os.path.join(INPUT_BG_DIRECTORY, name)) ]

# We scan all the files in the sub-directories with the extensions given above
all_bg_files = tf.concat(
    [tf.train.match_filenames_once(INPUT_BG_DIRECTORY + "/" + x + "/*.png") for x in bg_directories],
    0
)

all_bg_files_length = tf.shape(all_bg_files)[0]

### 2.2. Define the image reader

Read an entire image file which is required since they're JPEGs, if the images are too large they could be split in advance to smaller files or use the Fixed reader to split up the file.

In [4]:
image_reader = tf.WholeFileReader()

### 2.3. Read images from the Queue One by One
Read a whole file from the queue, the first returned value in the tuple is the filename which we are ignoring.

In [5]:
image_path, image_file = image_reader.read(filename_queue)

### 2.4. Convert each Image to a Tensor

Decode the image file, this will turn it into a Tensor which we can then use in training. It automatically detect whether the image is ["GIF", "PNG", "JPEG"] and which decoder to use.

In [6]:
def string_length_tf(t):
    return tf.py_func(lambda x: len(x), [t], tf.int32)

In [7]:
path_length = string_length_tf(image_path)
        
image_data  = tf.image.decode_png(image_file, channels=4)

image_label = tf.string_split([image_path] , delimiter=os.path.sep).values[1]  

## 3. Perform Image Augmentation

### 3.1 Define an Image Augmentation Function

In [8]:
def augment_image(image):
    
    ### GAMMA SHIFTING => It affects primarily the high lights ###
    
    random_gamma      = tf.random_uniform([], 0.5, 1.3)
    image_aug         = image ** random_gamma
    
    ### BRIGHTNESS SHIFTING ###
    
    # This gives a centered random  image*(1 +/- delta)
    # It does not fit our requirements, we would like a random brightness not centered around "1".
    #image = tf.image.random_brightness(image, max_delta=0.125) 
    
    random_brightness = tf.random_uniform([], 0.3, 1.2)
    image             =  image * random_brightness
    
    ### OPS SHIFTING ###   
    
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    
    # randomly horizontally flip the image
    do_flip = tf.random_uniform([], 0, 1)
    image  = tf.cond(do_flip > 0.5, lambda: tf.image.flip_left_right(image), lambda: image)
    
    # randomly rotate the image
    n_rot = tf.random_uniform([], 0, 3, tf.int32) # 0 => No Rotation, 1 => 90° Rot, 2 => 180° Rot, 3 => 270° Rotation
    image = tf.image.rot90(image, n_rot)
    
     # The random_* ops do not necessarily clamp.
    image = tf.clip_by_value(image, 0.0, 255.0)
    
    return tf.cast(image, tf.uint8)

### Create Background Replacing Functions

In [9]:
def color_to_alpha(image, luminance_threshold=70):
    
    data = np.array(image)
    
    rgb = data[:,:,:3]
    
    mask = list()

    for row in rgb:
        row_arr = list()

        for pix in row:
            lum = ((0.2126*pix[0]) + (0.7152*pix[1]) + (0.0722*pix[2]))/255*100
            row_arr.append(lum >= luminance_threshold)

        mask.append(row_arr)

    mask = np.array(mask)

    transparent_color = [0, 0, 0, 0]
    
    # change all pixels that match color to transparent_color*
    data[mask] = transparent_color
    
    return data

In [10]:
def overlay_image2background(image, bg_filename):
    bg = Image.open(bg_filename)
    bg = bg.convert("RGBA")
    
    composed_img = Image.alpha_composite(bg, Image.fromarray(image))
    
    return composed_img.convert("RGB")

In [11]:
def get_random_bg_filename():
    rand_idx = tf.random_uniform(
        [1], 
        minval = 0, 
        maxval = all_bg_files_length - 1,
        dtype  = tf.int32
    )[0]

    return all_bg_files[rand_idx]

### 3.2. Create a Tensor of Images and Populate it

In [12]:
image_transparent = tf.py_func(color_to_alpha, [image_data], tf.uint8)

In [13]:
img_arr = tf.stack([
    tf.image.encode_png(image_data),
])

for _ in range(IMG_AUGMENTATION_FACTOR):
    rand_idx = tf.random_uniform(
        [1], 
        minval = 0, 
        maxval = all_bg_files_length - 1,
        dtype  = tf.int32
    )[0]

    bg_filename = all_bg_files[rand_idx]
    img_with_random_bg       = tf.py_func(overlay_image2background, [image_transparent, get_random_bg_filename()], tf.uint8)
    img_with_random_bg_float = tf.cast(img_with_random_bg, tf.float32) 
    
    img_arr = tf.concat([img_arr, [tf.image.encode_png(augment_image(img_with_random_bg_float))]], 0)

## 4. Define an Initialisation Operation

In [14]:
init_op_global = tf.global_variables_initializer()
init_op_local = tf.local_variables_initializer()

## 5. Define a function generating random filenames

In [15]:
def id_generator(size=20, chars=string.ascii_uppercase + string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

## 7. Launch the dataset generation Session

In [16]:
with tf.Session() as sess:
    sess.run([init_op_global, init_op_local])

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    try:
        
        i = 0        
        n_files = len(all_files.eval())
        
        print("Number of Images to process %d\n" % n_files)
        
        while not coord.should_stop():
            
            _lbl_txt, _img_arr = sess.run([image_label, img_arr])   
            
            ## Increment ops count
            i += 1 

            out_dir = OUTPUT_DIRECTORY + "/" + _lbl_txt.decode("utf-8")
            
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
                 
            for _img in _img_arr:
                filename = out_dir + "/" + id_generator() + ".png"

                with open(filename, "wb+") as f:
                    f.write(_img)
                    f.close()
            
            if (i % 300 == 0):
                print ("Processing Image: %d/%d => %.2f%%" % (i, n_files, i/n_files*100))
            
    except tf.errors.OutOfRangeError:
        pass
    
    finally:        
        print("\nNumber of Images Processed: %d" % i)
        
        coord.request_stop()
        coord.join(threads)

Number of Images to process 7800

Processing Image: 300/7800 => 3.85%
Processing Image: 600/7800 => 7.69%
Processing Image: 900/7800 => 11.54%
Processing Image: 1200/7800 => 15.38%
Processing Image: 1500/7800 => 19.23%
Processing Image: 1800/7800 => 23.08%
Processing Image: 2100/7800 => 26.92%
Processing Image: 2400/7800 => 30.77%
Processing Image: 2700/7800 => 34.62%
Processing Image: 3000/7800 => 38.46%
Processing Image: 3300/7800 => 42.31%
Processing Image: 3600/7800 => 46.15%
Processing Image: 3900/7800 => 50.00%
Processing Image: 4200/7800 => 53.85%
Processing Image: 4500/7800 => 57.69%
Processing Image: 4800/7800 => 61.54%
Processing Image: 5100/7800 => 65.38%
Processing Image: 5400/7800 => 69.23%
Processing Image: 5700/7800 => 73.08%
Processing Image: 6000/7800 => 76.92%
Processing Image: 6300/7800 => 80.77%
Processing Image: 6600/7800 => 84.62%
Processing Image: 6900/7800 => 88.46%
Processing Image: 7200/7800 => 92.31%
Processing Image: 7500/7800 => 96.15%
Processing Image: 780